# resultsDB Query 

###  Kat Nykiel, Alejandro Strachan
School of Materials Engineering and Birck Nanotechnology Center, Purdue University, West Lafayette, Indiana 47907, United States

This notebook demonstrates how to query results of the wflowingestor tool, and perform analysis to extract additional features for machine learning. Here, we use a high-throughput dataset of hexagonal, layered carbides which have previously been loaded into the wflowingestor tool

## Load Sim2L Results
We start by querying resultsDB for all DTM_MAX simulations

In [ ]:
# Import libraries
import os
import pandas as pd
import numpy as np
import json
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pymatgen.core import Structure, Composition
from pymatgen.core.periodic_table import Element
from pymatgen.io.vasp.inputs import Poscar, Incar, Kpoints


# Import nanoHUB-specific libraries
import nanohubremote as nr
from simtool import findInstalledSimToolNotebooks,searchForSimTool
from simtool import getSimToolInputs,getSimToolOutputs,Run


In [ ]:
# Create a nanoHUB web services session
auth_data = {
    'grant_type' : 'tool',
}
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"]
    
session = nr.Sim2l(auth_data)

In [ ]:
# Query the vaspingestor tool for all runs submitted by Kat Nykiel with the DTM_MAX tag
tool = 'vaspingestor'

installedSimToolNotebooks = findInstalledSimToolNotebooks(tool,returnString=True)
print(installedSimToolNotebooks)
cellrelaxdft = searchForSimTool(tool)

req_json = session.requestPost('results/dbexplorer/search?simtool=true', data={ 'filters': '[{"field": "input.author", "operation": "=", "value": "Kat '
             'Nykiel"},{"field": "input.dataset", "operation": "=", "value": "DTM_MAX"}]',
  'limit': 10000,
  'results': '['
             '"output.XC_functional"]',
  'revision': 0,
  'tool': 'vaspingestor'}, timeout=20) # QUERY
req_json = req_json.json()
data = pd.DataFrame(req_json['results']).dropna().reset_index(drop=True)
print(f'data size: {data.shape[0]}')
squids = data['squid'].values
data.head()

In [ ]:
# Obtain Sim2L outputs, for all versions published. If we want a specific tool version, we can filter only for squids with a specific revision name, ex. 'r7'
req_jsons = []
 
for ids in np.array_split(squids,50):
    search = {
        'tool':'vaspingestor', 
        'filters':json.dumps([
            {'field':'squid','operation':'in','value': str(tuple(ids))},
        ]),
        'results':json.dumps([
            "output.structure", "output.composition", 
                 "output.lattice_parameters", "output.lattice_angles", 
                 "output.energy", "output.forces", "output.max_force", 
                 "output.rms_force", "output.KPOINTS", "output.ENCUT", 
                 "output.XC_functional"
        ]),
        'simtool' : 1,
        'limit' : 500
    }
    req_json = session.requestPost('results/dbexplorer/search', data=search)
    req_json = req_json.json()
    req_jsons.append(req_json)
    
req_dfs = []
for req_json in req_jsons:
    req_dfs.append(pd.DataFrame(req_json['results']))
results_df = pd.concat(req_dfs)

results_df.head()

## Extract additional features
Next, we extract additional features not stored in the Sim2L. These are features which either depend on outside simulations (formation energy, cohesive energy) or are MAX-specfic (bond lengths, interplanar distances)

In [ ]:
# Remove unneccesary columns, strip 'output.' from column names
results_df.drop('squid',axis=1,inplace=True)

# create a dictionary to map old column names to new column names
new_col_names = {col: col.replace('output.', '') for col in results_df.columns if col.startswith('output.')}

# rename columns using the dictionary created above
results_df.rename(columns=new_col_names, inplace=True)

results_df.head()

In [ ]:
def get_features(doc, e_df):
    """get set of extended features from Sim2L

    Args:
        doc (dict): row of pandas df from vaspingestor sim2L
        e_df (DataFrame): Dataframe containing formation/cohesive energy values
        
    Returns:
        features (dict): extracted feature dictionary
    """    

    features = {}
    
    # Build structure object
    struct = Structure.from_dict(doc['structure'])
    n_map = {8:1,12:2,16:3}
    n = n_map[struct.num_sites]
    
    # Formation and cohesive energies
    comp_df = pd.DataFrame.from_dict(doc['composition'], orient='index', columns=['n'])
    elements_df = e_df.set_index('element')
    E_form = (comp_df['n'] * elements_df.loc[comp_df.index]['formation_energy']).sum()
    E_coh = (comp_df['n'] * elements_df.loc[comp_df.index]['cohesive_energy']).sum()
        
    features['formation_energy_per_atom'] = (doc['energy'] - E_form)/struct.num_sites
    features['cohesive_energy_per_atom'] = (doc['energy'] - E_coh)/struct.num_sites
    
    # Bond lengths
    # r_MX
    sites = {1:[0,6],2:[5,10],3:[7,13]}
    features['r_MX'] = struct.get_distance(*sites[n])
    # r_MA
    sites = {1:[0,5],2:[3,6],3:[3,9]}
    features['r_MA'] = struct.get_distance(*sites[n])

    # Interplanar distances
    # d_AA
    sites = {1:[4,5],2:[6,7],3:[8,9]}
    features['d_AA'] = get_z_distance(struct, *sites[n])
    # d_MM
    sites = {1:[0,2],2:[2,3],3:[1,3]}
    features['d_MM'] = get_z_distance(struct, *sites[n])
    # d_MX
    sites = {1:[0,6],2:[5,10],3:[1,13]}
    features['d_MX'] = get_z_distance(struct, *sites[n])
    # d_XA
    sites = {1:[5,6],2:[6,11],3:[8,13]}
    features['d_XA'] = get_z_distance(struct, *sites[n])

    # Add a few more features for plotting
    features['c']=struct.lattice.abc[2]
    species= [site.specie for site in struct]
    uniques, i = np.unique(species,return_index=True)
    sort_i = sorted(i)
    elements = [s.symbol for s in [species[i] for i in sort_i]] 
    
    if len(elements) == 4:
        features['M1'] = elements[0]
        features['M2'] = elements[1]
        features['A'] = elements[2]
        features['X'] = elements[3]
    
    elif len(elements) == 3:
        features['M1'] = elements[0]
        features['M2'] = elements[0]
        features['A'] = elements[1]
        features['X'] = elements[2]
        
    features['n']=n

    return features

def get_z_distance(structure, site_idx1, site_idx2):
    # return distance along z axis betwen two sites in Structure object
    site1 = structure[site_idx1]
    site2 = structure[site_idx2]
    z_distance = abs(site1.coords[2] - site2.coords[2])
    return z_distance

with open('energies.csv') as f:
    e_df = pd.read_csv(f)

features = []

for index, doc in results_df.iterrows():
    features.append(get_features(doc, e_df))

feature_df = pd.DataFrame(features)
feature_df.head()


In [ ]:
# Add extracted features to results_df
DTM_df = pd.concat([results_df.reset_index(),feature_df.reset_index()],axis=1)
DTM_df.head()

## Visualize Dataset
Finally, we provide several plots to visualize the wide domain of this dataset

### Scatter plots

In [ ]:
# Generate 4 scatter plots of the data, colored by primary M', A, X elements and n number of layers
for i,color in enumerate(['n','M1','A','X',]):
    titles = ['n','M\'','A','X']
    fig=go.Figure()
    metals = DTM_df[color].unique()
    colors = px.colors.qualitative.Prism
    color_dict = dict(zip(metals,colors))
    for metal in metals:
        sdf = DTM_df[DTM_df[color]==metal]
        try:
            fig.add_trace(go.Scatter(x=sdf['c'],y=sdf['formation_energy_per_atom'], mode='markers',marker = {'color':color_dict[metal],'size':4}, name=str(metal))) #,col=i%2+1, row = i//2+1)
        except KeyError:
            print(f"Could not generate a plot for {metal}")
    fig.update_layout(
        xaxis_title='c (Å)',
        yaxis_title='formation energy (eV/atom)',
        # title=,
#         template='simple_white',
        width=600,
        height=600,
        font_size=20,
#         plot_bgcolor='white',
        legend=dict(x=.9,y=0.5,itemsizing='constant',title=titles[i])
    )
    fig.add_shape(type='line', x0=15.8, y0=-2.5, x1=15.8, y1=0.5, line=dict(dash='dot',color='black'))
    fig.add_shape(type='line', x0=21.1, y0=-2.5, x1=21.1, y1=0.5, line=dict(dash='dot',color='black'))
    fig.show()

### Violin plot

In [ ]:
fig = go.Figure()
metals = DTM_df.M1.unique()
means = []
for metal in metals:
    means.append(np.mean(DTM_df[DTM_df['M1']==metal].formation_energy_per_atom.values))
metals = [x for _, x in sorted(zip(means,metals))]
for metal in metals:
    fig.add_trace(go.Violin(x=DTM_df['M1'][DTM_df['M1']==metal][DTM_df['X']=='C'],y=DTM_df['formation_energy_per_atom'][DTM_df['M1']==metal][DTM_df['X']=='C'],name=f'{metal},C', side='positive',legendgroup='C',scalegroup='C',line_color='blue'))
    fig.add_trace(go.Violin(x=DTM_df['M1'][DTM_df['M1']==metal][DTM_df['X']=='N'],y=DTM_df['formation_energy_per_atom'][DTM_df['M1']==metal][DTM_df['X']=='N'],name=f'{metal},N', side='negative',legendgroup='N',scalegroup='N',line_color='orange'))
fig.update_traces(
    meanline_visible=False,
    points=False
)
fig.update_layout(
    violingap=0,
    violinmode='overlay',
    xaxis_title='M\' element',
    yaxis_title='formation energy (eV/atom)',
    # title='Formation energy vs. transition metal element',
    template='simple_white',
    showlegend=False,
    font_size=16,
    yaxis_range = [-3.5,1]
)
fig.show()

### Heatmaps

In [ ]:
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import iplot

X_list = ['C','N']
for X in X_list:

    fig = make_subplots(rows=3, cols = 4, subplot_titles = ["Al","Si","P","S","Ga","Ge","As","Cd","In","Sn","Tl","Pb"])

    for f,A in enumerate(["Al","Si","P","S","Ga","Ge","As","Cd","In","Sn","Tl","Pb"]):

        A_df = DTM_df[DTM_df.A==A]
        A_df = A_df[A_df.X==X]

        # create a pivot table that maps col2 values to each unique col1 value
        pivot = A_df.pivot_table(values='formation_energy_per_atom', index='M1', columns='M2')

        # get the unique values of col1 and col2 from the original dataframe
        x_labels = ["Sc","Ti","V","Cr","Mn","Zr","Nb","Mo","Hf","Ta","W"]
        y_labels = x_labels

        # create a list of dictionaries representing each row in the heatmap
        rows = []
        for i in range(len(y_labels)):
            row = {'y': y_labels[i]}
            for j in range(len(x_labels)):
                value = pivot.loc[y_labels[i], x_labels[j]]
                row[x_labels[j]] = value
            rows.append(row)

        # define the heatmap trace using the rows list and axis labels
        fig.add_trace(go.Heatmap(z=[list(row.values())[1:] for row in rows],
                        x=list(x_labels),
                        y=list(y_labels),
                        name=f'A={A}', coloraxis='coloraxis'),col = f%4+1, row = f//4+1)

    fig.for_each_annotation(lambda ann: ann.update(font=dict(size=22)))

    for i in range(3):
        for j in range(4):
            fig.update_xaxes(title_text='M\' element', row=i+1, col = j+1)
            fig.update_yaxes(title_text='M\'\' element', row=i+1, col=j+1)

    fig.update_layout(
        template='simple_white',
        xaxis_title='M\' element',
        yaxis_title='M\'\' element',
        width=1600,
        height=1000,
        title=f'formation energy for {X}-based MAX phases')
    fig.update_coloraxes(colorbar_title=dict(text='E<sub>form</sub>(eV/atom)'),colorbar_title_font_size=16)

    fig.show()

## Re-building input files from Sim2L
An important part of FAIR science is to provide the necessary information to re-create simulations. The vaspingestor Sim2L contains within it the information required to replicate our DTM MAX VASP workflow, and the example below demonstrates how to generate the INCAR, KPOINTS, and POSCAR files, in addition to specifying the required POTCAR files for VASP.

In [ ]:
# Query for an arbitrary result from the Sim2L database
req_json = session.requestPost('results/dbexplorer/search?simtool=true', data={ 'filters': '[{"field": "input.author", "operation": "=", "value": "Kat '
             'Nykiel"},{"field": "input.dataset", "operation": "=", "value": "DTM_MAX"}]',
  'limit': 1,
  'results': '['
             '"input.doc", "output.structure", "output.KPOINTS", "output.energy"]',
  'revision': 0,
  'tool': 'vaspingestor'}, timeout=20) # QUERY

req_json = req_json.json()
data = pd.DataFrame(req_json['results']).dropna().reset_index(drop=True)
run = data.iloc[0]
data.head()

### POSCAR file

In [ ]:
# Get structure object from results
structure = Structure.from_dict(run['output.structure'])

# Create a Poscar object from the Structure object
poscar = Poscar(structure)

# Write the POSCAR file to disk. Here, we just print it instead
# poscar.write_file('POSCAR')
print(poscar)

### INCAR file


In [ ]:
# Get Incar object from Sim2L (here, we use a single-point calculation)
incar_params = run['input.doc']['input']['parameters']

# Create an Incar object from the dictionary
incar = Incar.from_dict(incar_params)

# Write the INCAR file to disk
# incar.write_file('INCAR')
print(incar)

### KPOINTS file

In [ ]:
# Get Kpoints data
kpoints = run['output.KPOINTS']

# Create a Kpoints object
kpoints = Kpoints.gamma_automatic(kpts=kpoints, shift=(0, 0, 0))

# Write the KPOINTS file to disk
# kpoints.write_file('KPOINTS')
print(kpoints)

### POTCAR files

In [ ]:
# List the POTCAR specs used
run['input.doc']['input']['potcar_spec']

## Convex Hull

The code below demonstrates how to plot the ternary/quaternary convex hull for a given compound

In [ ]:
# Get API key
def read_key():
    """
    Read in new Materials Project API key
    """
    import os, stat
    from IPython.display import clear_output

    # Read in new Materials Project API key, if one exists
    try:
        with open(os.path.expanduser('~/.mpkey.txt'), 'r') as f:
            key = f.readlines()[0]
            return key
    except:
        key = ""

    # Check if API key already exists, skip try-except
    if not key:
        # Prompt user for API key
        try:
            user = str(input())
            clear_output()
            if not user.isalnum():
                raise TypeError('Wrong Key')
            if user == None:
                raise TypeError('Empty')
            with open(os.path.expanduser('~/.mpkey.txt'), 'w') as keyfile:
                keyfile.write(user)
            os.chmod(os.path.expanduser('~/.mpkey.txt'), stat.S_IREAD | stat.S_IWRITE)
            del user

            with open(os.path.expanduser('~/.mpkey.txt'),'r') as f:
                key = f.readlines()[0]
                return key
            print("Success")
        except:
            print("Something seems wrong with your key")
            
api_key = read_key()

In [ ]:
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter
from pymatgen.entries.computed_entries import ComputedEntry
from mp_api.client import MPRester

# Instantiate the client with your API key
mpr = MPRester(api_key)

# Pick a run from the dataframe. Here, we pick one with a relatively low E_form which lies above the convex hull
convex_hull_run = DTM_df.iloc[4]

# Get run details
structure = Structure.from_dict(convex_hull_run['structure'])
E_form = convex_hull_run['formation_energy_per_atom']*structure.num_sites

# Create a ComputedEntry object for the structure
entry = ComputedEntry(structure.composition, E_form)

chemsys = [str(e) for e in structure.composition.elements]

# Get the stable phases and their energies
entries = mpr.get_entries_in_chemsys(chemsys)

print(entry)

# Create a phase diagram object and add the entries
pdi = PhaseDiagram([entry] + entries)

# Plot phase diagram
plotter = PDPlotter(pdi)
plotter.show()

print(pdi)